##Documentação CSV

O formato CSV (Comma-Separated Values) é amplamente utilizado para armazenamento e intercâmbio de dados tabulares devido à sua simplicidade e compatibilidade com diversas ferramentas e plataformas. Nesta seção, exploraremos a documentação e os principais recursos para manipulação de arquivos CSV, especialmente no contexto do Databricks, facilitando a leitura, escrita e processamento eficiente desses dados em ambientes de análise e ciência de dados.

### Dados abertos

Pesquisar google dados gov dados publicos

https://dados.gov.br/dados/conjuntos-dados

Defesa e segurança Formato csv > Recursos

https://dados.gov.br/dados/conjuntos-dados/ocorrencias-aeronauticas

### Pesquisar csv documentation databricks

https://docs.databricks.com/pt/external-data/csv.html

https://spark.apache.org/docs/latest/sql-data-sources-csv.html

In [0]:
path = "/Volumes/workspace/default/tutorial/anac"
display(dbutils.fs.ls(path))

In [0]:
# Ler o arquivo CSV em um DataFrame  modo antigo sem tratamento
df_csv = spark.read.csv(f"{path}/V_OCORRENCIA_AMPLA.csv")
display(df_csv)

In [0]:
# Passando Opções avançadas
# Modo de Leitura do spark
# .format: Formato do Arquivo
# skipRows: Quantas Linhas Pular
# header: 1° linha é o Cabeçalho?
# sep: Separador do CSV
# inferSchema: Inserir os tipos de Dados automaticamente (se não colocar vai vir tudo string)
df = spark.read \
    .format("csv") \
    .option("skipRows", 1) \
    .option("header", True) \
    .option("sep", ";") \
    .option("inferSchema", True) \
    .load(f"{path}/V_OCORRENCIA_AMPLA.csv")

display(df)


### Ajustando Colunas do CSV

In [0]:
# CSV ContatoShar da aula anterior (3. Transformado Consulta de Tabela SQL em DataFlame)
caminho_csv = "/Volumes/workspace/bike_store/outputs/contact_sha.csv"
df_csv = spark.read.csv(caminho_csv, header=False, inferSchema=True)
display(df_csv)

In [0]:
# Renomenado colunas
df_csv = df_csv.withColumnRenamed("_c0", "id") \
               .withColumnRenamed("_c1", "name") \
               .withColumnRenamed("_c2", "phone") \
               .withColumnRenamed("_c3", "email")

display(df_csv)



## Salvando Arquivo em Tamanho reduzido

https://spark.apache.org/docs/latest/sql-data-sources-csv.html

Ao lidar com grandes volumes de dados em formato CSV, é comum a necessidade de reduzir o tamanho dos arquivos para otimizar o armazenamento e facilitar o processamento. Nesta seção, abordaremos estratégias e boas práticas para salvar arquivos CSV em tamanhos menores, mantendo a integridade dos dados e melhorando a eficiência em ambientes de análise, como o Databricks.

### Conversor de Bytes para Megabytes (MB)

Para converter bytes em megabytes (MB), basta dividir o valor em bytes por 1.048.576 (ou seja, 1024 × 1024):


MB = bytes / 1.048.576


**Exemplo:**  
Se um arquivo possui 5.242.880 bytes:


MB = 5.242.880 / 1.048.576 = 5 MB

In [0]:
# Salvando um DataFlame
df = spark.read \
    .format("csv") \
    .option("skipRows", 1) \
    .option("header", True) \
    .option("sep", ";") \
    .option("inferSchema", True) \
    .load(f"{path}/V_OCORRENCIA_AMPLA.csv")

display(df)

In [0]:
# Salvando em formato comprimido (melhor para questoes de armazenamento em nuvem onde paga pelo tanto que armazenar)
# write = Gravar\Escrever 
# overwrite = Sobrescrever o arquivo
df.write \
    .format("csv") \
    .option("compression", "gzip") \
    .option("header", "true") \
    .option("sep", ";") \
    .mode("overwrite") \
    .save(f"{path}/V_OCORRENCIA_AMPLA_CSV.zip")

In [0]:
display(dbutils.fs.ls(path))

In [0]:
# O CSV tem 9MB e o ZIP ficou com 2MB
def get_folder_size(path):
    files = dbutils.fs.ls(path)
    return sum(f.size for f in files if not f.isDir())

folder_size = get_folder_size(f"{path}/V_OCORRENCIA_AMPLA.csv")
print(f"Tamanho total da pasta CSV: {folder_size} bytes")

folder_size = get_folder_size(f"{path}/V_OCORRENCIA_AMPLA_CSV.zip")
print(f"Tamanho total da pasta ZIP: {folder_size} bytes")

##Lendo Arquivo salvo  em tamanho reduzido

Ao trabalhar com grandes volumes de dados, é fundamental garantir que os arquivos estejam otimizados para leitura e processamento. Após reduzir o tamanho dos arquivos CSV, o próximo passo é realizar a leitura desses arquivos de forma eficiente, mantendo a integridade e a qualidade das informações. Nesta seção, apresentaremos as principais práticas e comandos para ler arquivos CSV compactados ou otimizados no Databricks, facilitando a análise e o tratamento dos dados.

In [0]:
# read = ler fazer a leitura do arquivo
dfnovo = spark.read \
            .option("compression", "gzip") \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .option("sep", ";") \
            .csv(f"{path}/V_OCORRENCIA_AMPLA_CSV.zip")

display(dfnovo)